In [1]:
%matplotlib inline

In [2]:
path = '../data/redux/'
# path = '../data/redux/sample/'

In [3]:
from __future__ import division,print_function
import os
import json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
import matplotlib.pyplot as plt

In [4]:
import utils; reload(utils)
from utils import plots

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [5]:
batch_size = 64
# batch_size = 4

In [6]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [7]:
vgg = Vgg16()
batches = vgg.get_batches(path+'train',batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid',batch_size=batch_size)
vgg.finetune(batches)
vgg.fit(batches,val_batches,nb_epoch=1)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 658s - loss: 0.1195 - acc: 0.9686 - val_loss: 0.0853 - val_acc: 0.9795


In [9]:
vgg.model.save_weights(path+'results/ft1.h5')

#### Predictions

In [14]:
test_batches, test_preds =vgg.test(path+'test/')

Found 12500 images belonging to 1 classes.


In [15]:
filenames = test_batches.filenames

In [17]:
utils.save_array(path+'results/test_preds.dat',test_preds)
utils.save_array(path+'results/filenames.dat',filenames)

In [18]:
preds = utils.load_array(path+'results/test_preds.dat')
filenames = utils.load_array(path+'results/filenames.dat')

In [19]:
filenames[:5]

array(['unknown/9292.jpg', 'unknown/12026.jpg', 'unknown/9688.jpg', 'unknown/4392.jpg',
       'unknown/779.jpg'], 
      dtype='|S17')

In [21]:
preds[:5]

array([[  1.0000e+00,   1.7129e-11],
       [  9.5123e-01,   4.8765e-02],
       [  1.0000e+00,   1.1433e-07],
       [  1.0000e+00,   1.2510e-13],
       [  2.0968e-08,   1.0000e+00]], dtype=float32)

In [24]:
is_dog = preds[:,1]
is_dog[:5]

array([  1.7129e-11,   4.8765e-02,   1.1433e-07,   1.2510e-13,   1.0000e+00], dtype=float32)

In [23]:
ids = [int(f[8:f.find('.')]) for f in filenames]
ids[:5]

[9292, 12026, 9688, 4392, 779]

In [28]:
subm = np.stack([ids,is_dog],axis=1)
subm[:5]

array([[  9.2920e+03,   1.7129e-11],
       [  1.2026e+04,   4.8765e-02],
       [  9.6880e+03,   1.1433e-07],
       [  4.3920e+03,   1.2510e-13],
       [  7.7900e+02,   1.0000e+00]])

In [30]:
np.savetxt(path+'results/subm97.csv',subm,fmt='%d,%.5f',header='id,label',comments='')

IOError: [Errno 2] No such file or directory: 'redux/data/results/subm97.csv'

In [ ]:
from IPython.display import FileLink
FileLink(path+'results/subm97.csv')